## MonarQ's transpiler

MonarQ's transpiler is typically used behind the scene when you use the "monarq.default" device

In [ ]:
from pennylane_snowflurry.API.api_client import MonarqClient
from pennylane_snowflurry.transpiler.transpiler_config import MonarqDefaultConfig
from pennylane_snowflurry.transpiler.steps.placement import ISMAGS
import pennylane as qml
from dotenv import dotenv_values

In [ ]:
conf = dotenv_values(".env")
client = MonarqClient(conf["HOST"], conf["USER"], conf["ACCESS_TOKEN"])

dev = qml.device("monarq.default", shots=1000, client=client)

@qml.qnode(dev)
def circuit():
    qml.Hadamard(0)
    qml.CNOT([0, 1])
    qml.CNOT([1, 2])
    return qml.counts(wires=[0, 1, 2])

results = circuit()
print(results)

The usual behaviour of the transpiler is the following : 
1. decompose the circuit to the clifford + T gate set
2. map the circuit to the machine's topology using a pathfinding heuristic
3. route the unconnected 2 qubit gates using swaps
4. optimize by decomposing iteratively to RX, RZ and CZ, commuting, merging and cancelling inverses and trivial gates
5. decomposing to MonarQ's native gate set

It is possible to change the behaviour of the transpiler by changing steps, or adding new ones

In [ ]:
my_config = MonarqDefaultConfig() # this is the usual behaviour

ismags_placement = ISMAGS() # this is an alternative placement method that uses subgraph isomorphism

my_config.steps[1] = ismags_placement # the placement step is 2nd. change it for the new one

dev = qml.device("monarq.default", shots=1000, client=client, behaviour_config=my_config) # set the config explicitely

All steps that are played by default are **preprocessing** steps, but you can also add **postprocessing** steps, that act on the results instead of the circuit.

The steps will be filtered according to their type automatically.

In [ ]:
from pennylane_snowflurry.transpiler.steps.readout_error_mitigation import ReadoutErrorMitigation

readout_error_mitigation = ReadoutErrorMitigation()

my_config.steps.append(readout_error_mitigation)

You can create new preprocessing / postprocessing steps by overriding the PreProcessing / PostProcessing classes.

In [ ]:
# abstract steps + empty config
from pennylane_snowflurry.transpiler.steps.interfaces.pre_processing import PreProcStep
from pennylane_snowflurry.transpiler.steps.interfaces.post_processing import PostProcStep
from pennylane_snowflurry.transpiler.transpiler_config import TranspilerConfig

# default steps
from pennylane_snowflurry.transpiler.steps.base_decomposition import CliffordTDecomposition
from pennylane_snowflurry.transpiler.steps.placement import ASTAR
from pennylane_snowflurry.transpiler.steps.routing import Swaps
from pennylane_snowflurry.transpiler.steps.optimization import IterativeCommuteAndMerge
from pennylane_snowflurry.transpiler.steps.native_decomposition import MonarqDecomposition

In [ ]:
# toy preprocessing step for printing the circuit
class PrintCircuit(PreProcStep):
    def execute(self, tape):
        print([op.name for op in tape.operations])
        
# toy postprocessing step for printing the results
class PrintResults(PostProcStep):
    def execute(self, tape, results):
        print(results)

In [ ]:
# this custom config will print the circuit, transpile, print the transpiled circuit 
# and then print the unmitigated results, followed by the mitigated results.
my_config = TranspilerConfig(PrintCircuit(),
                             CliffordTDecomposition(), 
                             ASTAR(),
                             Swaps(),
                             IterativeCommuteAndMerge(),
                             MonarqDecomposition(), 
                             PrintCircuit(),
                             PrintResults(),
                             ReadoutErrorMitigation(), 
                             PrintResults())


You can also use the transpiler in a standalone manner if need be. 

In [ ]:
from pennylane_snowflurry.transpiler.monarq_transpile import Transpiler
from pennylane.tape import QuantumTape

In [5]:
# create a QuantumTape (this is one way to do it, but you can also retrieve it from a QNode)

ops = [qml.Hadamard(0), qml.CNOT([0, 1])]
measurements = [qml.counts(wires=[0, 1])]
shots = 1000

tape = QuantumTape(ops=ops, measurements=measurements, shots=shots)

tape.operations


[H(0), CNOT(wires=[0, 1])]

In [6]:
# configure and run the transpiler

config = MonarqDefaultConfig(use_benchmark=False)
transpiler = Transpiler.get_transpiler(config)

# since the transpiler function is a transform, you have to index [0][0] in order to get the resulting tape
tape = transpiler(tape)[0][0]

tape.operations

[X90(wires=[0]), X90(wires=[4]), Z(4), CZ(wires=[0, 4]), X90(wires=[4])]

## Steps

The default transpiling process contained in ```MonarqDefaultConfig``` contains the following steps :
1. **decompose** the circuit operations in a subset of operations (for simplifying subsequent operations)
2. **map** the circuit to physical qubits in MonarQ's topology
3. **route** 2 qubits gates which are not connected with respect to the mapping
4. **simplify** the circuit by commuting gates, merging rotations and cancelling inverses and trivial gates
5. **decompose** the circuit a subset of operations which are native to MonarQ

All transpiling steps reside in ```pennylane_snowflurry.transpiler.steps.*```

Steps are defined using a base class called ```BaseStep```. 

This class has two children, which subdivide the steps in two categories : 
- ```PreProcessing``` : steps that are applied before execution on the machine. Typically affects the circuit (through the ```QuantumTape``` class)
- ```PostProcessing``` : steps that are applied after execution on the machine. Typically affects the results.

It is possible to define new transpiling or postprocessing steps by overriding the ```PreProcessing```or ```PostProcessing``` classes as so : 

In [ ]:
from pennylane_snowflurry.transpiler.steps.interfaces.pre_processing import PreProcStep
from pennylane_snowflurry.transpiler.steps.interfaces.post_processing import PostProcStep

# this step will be executed in the transpiler
class MyPreProcessingStep(PreProcStep):
    def execute(self, tape):
        # define your own behaviour here
        return super().execute(tape) # return your modified tape here

# this step will be executed in the post-processor
class MyPostProcessingStep(PostProcStep):

    def execute(self, tape : QuantumTape, results : dict[str, int]):
        return super().execute(tape, results)